In [1]:
!which python

/Users/pushkalm11/miniconda3/envs/unrolling/bin/python


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power as MP

In [3]:
import random
import scipy
from scipy.spatial.distance import cdist
from scipy.io import savemat, loadmat

In [4]:
import networkx as nx
import sys

In [5]:
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Flatten, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping

In [6]:
print(f"TF Version: {tf.__version__}")

TF Version: 2.14.0


In [7]:
X_train = loadmat('./Dataset/X_train.mat')['X_Ground']
N, T = X_train.shape

In [8]:
dt = np.zeros([T - 1, T])

for i in range(T - 1):
    dt[i, i] = -1
    dt[i, i - 1] = 1

d = dt.T
d = np.concatenate((d, np.zeros([T, 1])), axis = 1) 
d[T - 1, T - 1] = -1
d = d.T
d[T - 1, T - 1] = 1 
d[-1, -2] = 1
d[0, -1] = 0
d = tf.convert_to_tensor(d, dtype = "float32")
print(d)

tf.Tensor(
[[-1.  0.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 [ 0.  1. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  0.  1.  1.]], shape=(500, 500), dtype=float32)


In [9]:
def adj_to_laplacian(AdjL):
    tmp = np.array(AdjL, dtype = "float32")
    return np.diag(np.sum(tmp, axis = 0)) - tmp

In [10]:
def initial_graph(X_training, N, top_values):
    # top_values = 6
    distance_matrix = cdist(X_training, X_training, metric = 'euclidean')
    sorted_indices = distance_matrix.argsort(axis = 1)
    
    TAdj = np.zeros([N, N])
    
    for i in range(N):
        for j in range(1, top_values):
            k = sorted_indices[i][j]
            TAdj[i][k] = 1
            
    TAdj = (TAdj + TAdj.T) / 2
    TAdj = np.where(TAdj > 0.1, 1, 0)
    AdjL = np.diag(np.sum(TAdj, 1)) - TAdj
    
    return AdjL

In [11]:
psi_cost = 0
psi_cost_1 = 0

In [12]:
# A - Predicted, B - Target matrix
def costfunc(A, B):
    # (y_true, y_pred)
    # print(A.shape)
    # print(B.shape)
    return (tf.norm(tf.multiply(psi_cost, (A - B)))) ** 2

# def costfunc(A, B):
#     return (tf.norm(tf.multiply(psi_cost, (A - B))) ** 2) + (reg * tf.norm(HLG @ B @ GLT) ** 2 / 2)

def mse_unknown(A, B):

    global psi_cost
    psi = psi_cost

    psid = tf.convert_to_tensor(np.ones([N, T], dtype = float) - psi, dtype = "float32")
    mse_unk = (LA.norm(np.multiply(psid, (X_train - B)))) ** 2 / np.sum(psid.numpy().flatten())
    
    global mse_unknown_list
    mse_unknown_list.append(mse_unk)
    
    return mse_unk

def mse_known(A, B):

    global psi_cost 
    psi = psi_cost

    mse_kn = (LA.norm(np.multiply(psi, (X_train - B)))) ** 2 / np.sum(psi.numpy().flatten())
    
    global mse_known_list
    mse_known_list.append(mse_kn)

    return mse_kn

In [13]:
callb = EarlyStopping(monitor = 'loss', mode = 'min', min_delta = 0.00001, verbose = 1, patience = 10)

def scheduler(epoch, lr):
    if epoch < 15:
        return lr
    else:
        return lr / lr_decay_factor

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [14]:
test_list = []

In [15]:
class EldarModel(tf.keras.Model):
    
    def __init__(self, Laplacian, reg_init, loops_init, d_init, **kwargs):
        super(EldarModel, self).__init__(**kwargs)
        p_init = 15.0
        self.p1_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p1 = tf.Variable(initial_value = self.p1_init, trainable = True, name = "p1")
        self.p2_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p2 = tf.Variable(initial_value = self.p2_init, trainable = True, name = "p2")
        self.p3_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p3 = tf.Variable(initial_value = self.p3_init, trainable = True, name = "p3")
        self.p4_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p4 = tf.Variable(initial_value = self.p4_init, trainable = True, name = "p4")
        
        self.lap = Laplacian
        self.reg = reg_init
        self.loops = loops_init
        self.d_matrix = d_init
        
    def call(self, y):
        
        psi = y[0, :, T:]
        global psi_cost
        psi_cost = psi
        
        psi = tf.convert_to_tensor(psi, dtype = "float32")
        
        X_in = y[0, :, : T]
        X_in = tf.convert_to_tensor(X_in, dtype = "float32")
        
        Y = X_in
        
        LG, reg, loop, d = self.lap, self.reg, self.loops, self.d_matrix
        
        LT = tf.matmul(tf.transpose(d), d)
        LT = tf.convert_to_tensor(LT, dtype = "float32")
        
        HLG = tf.eye(tf.shape(LG)[0], dtype = "float32") 
        GLT = LT + (self.p1 * MP(LT, 2)) + (self.p2 * MP(LT, 3)) + (self.p3 * MP(LT, 4)) + (self.p4 * MP(LT, 5))
        
        Xk = tf.zeros_like(X_in)
        Zk = tf.zeros_like(X_in)
        Zk = -(tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT)))
        
        for i in range(loop):
            # print(tf.multiply(psi, Xk) - Y, (HLG @ Xk @ GLT))
            fdx_xk = tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT))
            fdx_zk = tf.multiply(psi, Zk) - Y + (reg * (HLG @ Zk @ GLT))
            
            tau = tf.linalg.trace(tf.transpose(fdx_xk) @ Zk) / tf.linalg.trace(tf.transpose((Y + fdx_zk)) @ Zk)
            
            Xk_1 = Xk - (tau * Zk)
            
            fdx_xk_1 = tf.multiply(psi, Xk_1) - Y + (reg * (HLG @ Xk_1 @ GLT))
            
            gamma = (tf.norm(fdx_xk_1) ** 2) / (tf.norm(fdx_xk) ** 2)
            
            Zk_1 = (gamma * Zk) - fdx_xk_1
            
            Xk = Xk_1
            Zk = Zk_1
        
        global test_list
        test_list.append(Xk_1.numpy())
        
        return tf.reshape(Xk_1, [1, N, T])

In [16]:
no_of_psi = 10
Laplacian = initial_graph(X_train, N, 12)

# sensing_ratio = [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5]
sensing_ratio = (1 + np.arange(80)) / 100.0

# reg_list = np.load("./Outputs/eldar_KNN_reg.npy")
# loops_list = np.load("./Outputs/eldar_KNN_loops.npy")

reg_list = [1.0e-5] * len(sensing_ratio)
loops_list = [50] * len(sensing_ratio)

all_psi = []

mse_known_list = []
mse_unknown_list = []

learned_variables = []

lr_decay_factor = 1.05

In [17]:
for i_sen, rem in enumerate(sensing_ratio):
    
    print(f"\n\n###########################################################################################")
    print(f"################################### Sensing Ratio: {rem * 100}% ###################################")
    print(f"###########################################################################################\n\n")
    
    if i_sen == 0:
        M = int(rem * T)
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2*T])
    
        for i in range(no_of_psi):
            psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
            np.random.shuffle(psi_k)
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
        
        psi_cost_1 = psi_k
    
    else:
        previous_psi = all_psi[-no_of_psi : ]
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])
        
        rem_diff = int(N * (rem - sensing_ratio[i_sen - 1]) * T)
        
        for i in range(no_of_psi):
            psi_k_tmp = np.array(previous_psi[i]).flatten()
            ones_idx = np.where(psi_k_tmp == 1)
            
            ones_to_zero_idx = np.random.choice(ones_idx[0], size = rem_diff)
            psi_k_tmp[ones_to_zero_idx] = 0
            psi_k = psi_k_tmp
            
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
        
    reg_init = reg_list[i_sen]
    loops_init = loops_list[i_sen]
    
    sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 1.0e-3, momentum = 0.0)
    
    eldar_model = EldarModel(Laplacian, reg_init, loops_init, d, name = 'EM')
    eldar_model.compile(optimizer = sgd_optimizer, loss = costfunc, 
                        metrics = [mse_known, mse_unknown], run_eagerly = True)
    eldar_model.fit(X_train_concatenated, X_train_missing, epochs = 100, 
                    callbacks = [lr_scheduler], batch_size = 1)
    eldar_model.summary()
    
    all_variables = eldar_model.variables
    
    graph_vars = [i.numpy()[0, 0] for i in all_variables]
    learned_variables.append(graph_vars)
    
    print(f"Learned Variables: {graph_vars}")



###########################################################################################
################################### Sensing Ratio: 1.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 847ms/step - loss: 1.7333 - mse_known: 1.7508e-04 - mse_unknown: 0.0909 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 859ms/step - loss: 1.7331 - mse_known: 1.7506e-04 - mse_unknown: 0.0909 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 8s 843ms/step - loss: 1.7329 - mse_known: 1.7504e-04 - mse_unknown: 0.0909 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 848ms/step - loss: 1.7326 - mse_known: 1.7501e-04 - mse_unknown: 0.0909 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 8s 820ms/step - loss: 1.7324 - mse_known: 1.7499e-04 - mse_unknown: 0.0909 - lr: 0.0010
Epoch 6/100
10/

Learned Variables: [14.999038, 14.996346, 14.986299, 14.949253]


###########################################################################################
################################### Sensing Ratio: 2.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 882ms/step - loss: 1.6789 - mse_known: 1.7130e-04 - mse_unknown: 0.0951 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 8s 807ms/step - loss: 1.6787 - mse_known: 1.7128e-04 - mse_unknown: 0.0951 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 865ms/step - loss: 1.6785 - mse_known: 1.7126e-04 - mse_unknown: 0.0951 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 871ms/step - loss: 1.6783 - mse_known: 1.7124e-04 - mse_unknown: 0.0951 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 854ms/step - loss: 1.6780 - mse_know

Learned Variables: [14.999048, 14.996395, 14.9867, 14.95088]


###########################################################################################
################################### Sensing Ratio: 3.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 8s 813ms/step - loss: 1.6165 - mse_known: 1.6661e-04 - mse_unknown: 0.0982 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 8s 825ms/step - loss: 1.6163 - mse_known: 1.6659e-04 - mse_unknown: 0.0982 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 8s 804ms/step - loss: 1.6161 - mse_known: 1.6657e-04 - mse_unknown: 0.0982 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 8s 833ms/step - loss: 1.6159 - mse_known: 1.6655e-04 - mse_unknown: 0.0982 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 8s 805ms/step - loss: 1.6157 - mse_known: 

Learned Variables: [14.999066, 14.996461, 14.98712, 14.95269]


###########################################################################################
################################### Sensing Ratio: 4.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 8s 823ms/step - loss: 1.5658 - mse_known: 1.6305e-04 - mse_unknown: 0.0966 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 8s 817ms/step - loss: 1.5656 - mse_known: 1.6303e-04 - mse_unknown: 0.0966 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 8s 830ms/step - loss: 1.5654 - mse_known: 1.6301e-04 - mse_unknown: 0.0966 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 8s 819ms/step - loss: 1.5652 - mse_known: 1.6299e-04 - mse_unknown: 0.0966 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 8s 821ms/step - loss: 1.5651 - mse_known:

Learned Variables: [14.999075, 14.996635, 14.987539, 14.954126]


###########################################################################################
################################### Sensing Ratio: 5.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 942ms/step - loss: 1.5116 - mse_known: 1.5905e-04 - mse_unknown: 0.0983 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 8s 821ms/step - loss: 1.5114 - mse_known: 1.5903e-04 - mse_unknown: 0.0983 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 871ms/step - loss: 1.5112 - mse_known: 1.5902e-04 - mse_unknown: 0.0983 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 882ms/step - loss: 1.5110 - mse_known: 1.5900e-04 - mse_unknown: 0.0983 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 8s 821ms/step - loss: 1.5109 - mse_know

...

Learned Variables: [15.0, 15.0, 15.0, 14.999753]


###########################################################################################
################################### Sensing Ratio: 76.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 904ms/step - loss: 0.0098 - mse_known: 3.9361e-06 - mse_unknown: 0.2021 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 855ms/step - loss: 0.0098 - mse_known: 3.9425e-06 - mse_unknown: 0.2021 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 859ms/step - loss: 0.0098 - mse_known: 3.9397e-06 - mse_unknown: 0.2021 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 885ms/step - loss: 0.0098 - mse_known: 3.9397e-06 - mse_unknown: 0.2021 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 867ms/step - loss: 0.0098 - mse_known: 3.9410e-06 

Learned Variables: [15.0, 15.0, 15.0, 14.999779]


###########################################################################################
################################### Sensing Ratio: 77.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 888ms/step - loss: 0.0084 - mse_known: 3.5390e-06 - mse_unknown: 0.2030 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 895ms/step - loss: 0.0084 - mse_known: 3.5434e-06 - mse_unknown: 0.2030 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 874ms/step - loss: 0.0084 - mse_known: 3.5373e-06 - mse_unknown: 0.2030 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 872ms/step - loss: 0.0084 - mse_known: 3.5392e-06 - mse_unknown: 0.2030 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 857ms/step - loss: 0.0084 - mse_known: 3.5453e-06 

Learned Variables: [15.0, 15.0, 15.0, 14.999809]


###########################################################################################
################################### Sensing Ratio: 78.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 878ms/step - loss: 0.0071 - mse_known: 3.1066e-06 - mse_unknown: 0.2039 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 877ms/step - loss: 0.0071 - mse_known: 3.1202e-06 - mse_unknown: 0.2039 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 898ms/step - loss: 0.0071 - mse_known: 3.1142e-06 - mse_unknown: 0.2039 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 874ms/step - loss: 0.0071 - mse_known: 3.1082e-06 - mse_unknown: 0.2039 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 889ms/step - loss: 0.0071 - mse_known: 3.1085e-06 

Learned Variables: [15.0, 15.0, 15.0, 14.999892]


###########################################################################################
################################### Sensing Ratio: 79.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 864ms/step - loss: 0.0058 - mse_known: 2.6314e-06 - mse_unknown: 0.2044 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 856ms/step - loss: 0.0057 - mse_known: 2.6260e-06 - mse_unknown: 0.2044 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 880ms/step - loss: 0.0058 - mse_known: 2.6343e-06 - mse_unknown: 0.2044 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 875ms/step - loss: 0.0058 - mse_known: 2.6371e-06 - mse_unknown: 0.2044 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 908ms/step - loss: 0.0058 - mse_known: 2.6331e-06 

Learned Variables: [15.0, 15.0, 15.0, 14.999965]


###########################################################################################
################################### Sensing Ratio: 80.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 877ms/step - loss: 0.0052 - mse_known: 2.4848e-06 - mse_unknown: 0.2057 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 901ms/step - loss: 0.0052 - mse_known: 2.4692e-06 - mse_unknown: 0.2057 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 907ms/step - loss: 0.0052 - mse_known: 2.4688e-06 - mse_unknown: 0.2057 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 899ms/step - loss: 0.0052 - mse_known: 2.4788e-06 - mse_unknown: 0.2057 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 901ms/step - loss: 0.0052 - mse_known: 2.4766e-06 

In [18]:
for i, rem in enumerate(sensing_ratio):
    print(f"For {int(100 * rem)}% sensing ratio: {learned_variables[i]}")

For 1% sensing ratio: [14.999038, 14.996346, 14.986299, 14.949253]
For 2% sensing ratio: [14.999048, 14.996395, 14.9867, 14.95088]
For 3% sensing ratio: [14.999066, 14.996461, 14.98712, 14.95269]
For 4% sensing ratio: [14.999075, 14.996635, 14.987539, 14.954126]
For 5% sensing ratio: [14.999097, 14.996715, 14.987944, 14.955748]
For 6% sensing ratio: [14.999107, 14.996778, 14.988313, 14.957215]
For 7% sensing ratio: [14.999125, 14.996906, 14.988707, 14.9588]
For 8% sensing ratio: [14.999138, 14.997026, 14.989058, 14.9602]
For 9% sensing ratio: [14.999154, 14.997065, 14.989374, 14.961412]
For 10% sensing ratio: [14.999238, 14.997145, 14.989742, 14.962915]
For 11% sensing ratio: [14.999309, 14.99728, 14.9901085, 14.964332]
For 12% sensing ratio: [14.99933, 14.997368, 14.990437, 14.965653]
For 13% sensing ratio: [14.999339, 14.997419, 14.990788, 14.96695]
For 14% sensing ratio: [14.9993515, 14.997499, 14.991137, 14.968299]
For 15% sensing ratio: [14.999363, 14.997632, 14.991486, 14.969616]

In [19]:
np.save('./Outputs/eldar_KNN_mse_known.npy', mse_known_list)
np.save('./Outputs/eldar_KNN_mse_unknown.npy', mse_unknown_list)